[Project Euler 31: Coin sums](https://projecteuler.net/problem=31)

The idea is to iteratively count the number of descending sequences of coin values that sum to at most 200, without using 1p coins.  Since we can pad out any such sequence with 0 or more 1p coins to reach a sum of exactly 200, this will coincide with the desired number of ways to make change.

We use a DP approach.  On iteration $i$, `running_count` is the number of ways to make change for `max_value` with at most `i-1` coins, and `tally[(i,j)]` is the number of ways to get a sum of `i` using a collection of `i-1` coins whose smallest denomination is `j`.

In [1]:
import collections

def ways_to_get_sum(max_value):
    coin_values = (2, 5, 10, 20, 50, 100, 200)
    tally = collections.defaultdict(int)
    tally[(0, float('inf'))] = 1
    new_tally = None
    running_count = 0
    while any(tally):
        new_tally = collections.defaultdict(int)
        for (coin_sum, smallest_coin) in tally:
            running_count += tally[(coin_sum, smallest_coin)]
            for next_coin in coin_values:
                if next_coin <= smallest_coin and coin_sum + next_coin <= max_value:
                    new_tally[(coin_sum + next_coin, next_coin)] += tally[(coin_sum, smallest_coin)]
        tally = new_tally
    return(running_count)

In [2]:
%%time
ways_to_get_sum(200)

CPU times: user 8.63 ms, sys: 217 µs, total: 8.85 ms
Wall time: 11 ms


73682

A solution of mine from 2018 was the same idea but with a simple recursion.  It's prettier, but takes time exponential in the desired sum.

In [3]:
def ways_old(n,d):
    if n in {0,1} or d == 1:
        return 1
    else:
        wlist = []
        for k in [1,2,5,10,20,50,100,200]:
            if k <= n and k <= d:
                wlist.append(ways_old(n-k,k))
    return sum(wlist)

In [4]:
%%time
ways_old(200,200)

CPU times: user 78.7 ms, sys: 2.8 ms, total: 81.5 ms
Wall time: 83.6 ms


73682

Comparing these approaches:

In [5]:
%%time
ways_to_get_sum(1000)

CPU times: user 219 ms, sys: 2.99 ms, total: 222 ms
Wall time: 228 ms


321335886

In [6]:
%%time
ways_old(1000,200)

CPU times: user 4min 59s, sys: 1.71 s, total: 5min
Wall time: 5min 10s


321335886

So the "fast" way is better, but note that it's actually pretty bad, $O(n^2)$ in the desired sum.  We can improve this with a DP that iterates along desired sums instead of number of coins used.  In the code below, we construct a list of lists `tallies`, so that `tallies[i][j]` is the number of ways to make change for `i` pence with (any number of) coins whose largest denomination is the one at index `j` in the ordered list of denominations (except for the case of `tallies[0]`, which is defined to get the induction off the ground).  The result is $O(mn)$ in time/space complexity where $m$ is the number of denominations used.

In [7]:
def ways_to_get_better(desired_sum, list_of_denominations):
    list_of_denominations.sort()
    tallies = [[1]+[0]*(len(list_of_denominations)-1)]
    while len(tallies) <= desired_sum:
        current_tally = []
        for coin_index, coin_value in enumerate(list_of_denominations):
            current_sum = len(tallies)
            current_tally.append(0)
            if coin_value <= current_sum:
                for previous_coin_index, previous_coin_value in enumerate(list_of_denominations[:coin_index+1]):
                    current_tally[-1] += tallies[current_sum - coin_value][previous_coin_index]
        tallies.append(current_tally)
    return sum(tallies[-1])

In [8]:
%%time
ways_to_get_better(200, [1,2,5,10,20,50,100,200])

CPU times: user 1.76 ms, sys: 77 µs, total: 1.84 ms
Wall time: 2.2 ms


73682

In [9]:
%%time
ways_to_get_better(100000, [1,2,5,10,20,50,100,200])

CPU times: user 1.01 s, sys: 28.4 ms, total: 1.04 s
Wall time: 1.05 s


10056050940818192726001

Compare the previous "fast" method:

In [10]:
%%time
ways_to_get_sum(100000)

CPU times: user 1h 1min 4s, sys: 47.4 s, total: 1h 1min 51s
Wall time: 1h 16min 52s


10056050940818192726001

Woof!

The better way is indeed linear time:

In [11]:
%%time
ways_to_get_better(2000000, [1,2,5,10,20,50,100,200])

CPU times: user 20.6 s, sys: 445 ms, total: 21 s
Wall time: 21.1 s


12707037135786778764112854520001

Notice we only ever need to look at the last 200 indices of `tallies` to append its next element.  So we can modify to use a length 200 list that uses pointers to mimic a linked list-type structure, reducing space used to $O(md)$ ($m$ is the number of different denominations and $d$ is the maximum denomination &mdash; hence constant space when denominations are fixed).

In [12]:
def ways_to_get_best(desired_sum, list_of_denominations):
    list_of_denominations.sort()
    max_denom = list_of_denominations[-1]
    tallies = [[] for _ in range(max_denom+1)]
    tallies[0] = [1]+[0]*(len(list_of_denominations)-1)
    current_sum = 1
    while current_sum <= desired_sum:
        current_tally = []
        tally_index = current_sum % len(tallies)
        for coin_index, coin_value in enumerate(list_of_denominations):
            current_tally.append(0)
            if coin_value <= current_sum:
                for previous_coin_index, previous_coin_value in enumerate(list_of_denominations[:coin_index+1]):
                    current_tally[-1] += tallies[(current_sum - coin_value) % len(tallies)][previous_coin_index]
        tallies[tally_index] = current_tally
        current_sum += 1
    return sum(tallies[(current_sum-1) % len(tallies)])

In [13]:
%%time
ways_to_get_best(200, [1,2,5,10,20,50,100,200])

CPU times: user 1.58 ms, sys: 16 µs, total: 1.6 ms
Wall time: 1.62 ms


73682

In [14]:
%%time
ways_to_get_best(2000000, [1,2,5,10,20,50,100,200])

CPU times: user 22.3 s, sys: 49.4 ms, total: 22.4 s
Wall time: 22.5 s


12707037135786778764112854520001